# How to use an underwater acoustic propagation modeling with arlpy and Bellhop ?

The underwater acoustic propagation modeling toolbox (`uwapm`) in `arlpy` is integrated with the popular Bellhop ray tracer distributed as part of the [acoustics toolbox](https://oalib-acoustics.org/). In this notebook, we see how to use `arlpy.uwapm` to simplify the use of Bellhop for modeling.

- ## Prerequisites

    - Install [arlpy](https://pypi.org/project/arlpy/) (v1.6 or higher)

    You can install arlpy (in overall system) by typing this in your command prompt:
    ```bash
    python3 -m pip install arlpy
    ```
    - Install the [acoustics toolbox](https://oalib-acoustics.org/) (6 July 2020 version or later)

    You can use [this installation instructions](https://github.com/patel999jay/Bellhop-ARLPY-ECED6575) to install acoustics toolbox in overall system.

## Bellhop - Acoustic Toolbox

**What is BELLHOP ?**

- **BELLHOP** is a beam tracing model for predicting acoustic pressure fields in ocean environments.
- **BELLHOP** can produce a variety of useful outputs including transmission loss, eigenrays, arrivals, and received time-series. It also allows for range-dependence in the top and bottom boundaries (altimetry and bathymetry), as well as in the sound speed profile (SSP).
- **BELLHOP** is implemented in Fortran, Matlab, and Python and used on multiple platforms (Mac, Windows, and Linux).

# WHY BELLHOP?

- Underwater communication channel is a relatively difficult transmission medium due to the variability of link quality depending on location and applications. 
- Before deploying any kind of vehicles underwater, one should predict the underwater communication system performance which is based on the sound frequency transmitted underwater. 

**Why do you need to analyze the uw-comms performance?** 

- To analyze impact of channel characteristics on underwater communications,
- prior to deploying robots, predict communication system performance,
- provide guidance on best physical layout to deploy underwater vehicles,
- provide estimates on parameters for link budget calculation,

- **Because** it will provide you a rough idea about **how far you can communicate within network** which is also known as an **operation range for communication**. 

- **BELLHOP** reads these files depending on options selected within the main environmental file.

- i.e. Below is the example of the environmental file(*.env file).
    

- There are various options for which you can run bellhop are: **(That can be found in the *.env file as RUN TYPE)**
    - ray tracing option (R),
    - eigenray option (E),
    - transmission loss option ,
        - Coherent TL calculations (C)
        - Incoherent TL calculations (I)
        - Semi-coherent TL calculations (S)
    - an arrivals calculation option in ascii (A) ; an arrivals calculation option in binary (a)

# Sound Speed Profile of ANTARES

<h3><center> Figure 2: Sound Speed Profile </center></h3>

In [1]:
pip freeze

anyio==4.1.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arlpy==1.8.4
arrow==1.3.0
astropy==6.0.0
astropy-iers-data==0.2023.12.18.0.30.18
asttokens==2.4.1
async-lru==2.0.4
attrs==23.1.0
Babel==2.13.1
beautifulsoup4==4.12.2
bleach==6.1.0
blinker==1.4
bokeh==3.3.2
Cartopy==0.22.0
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
comm==0.2.0
command-not-found==0.3
contourpy==1.1.1
cryptography==3.4.8
cycler==0.11.0
dbus-python==1.2.18
debugpy==1.8.0
decorator==5.1.1
defusedxml==0.7.1
distlib==0.3.4
distro==1.7.0
distro-info==1.1+ubuntu0.1
ephem==4.1.5
et-xmlfile==1.1.0
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.0
filelock==3.6.0
fonttools==4.42.1
fqdn==1.5.1
healpy==1.16.6
httplib2==0.20.2
idna==3.6
importlib-metadata==4.6.4
ipykernel==6.27.1
ipython==8.18.1
ipywidgets==8.1.1
isoduration==20.11.0
jedi==0.19.1
jeepney==0.7.1
Jinja2==3.1.2
jplephem==2.21
json5==0.9.14
jsonpointer==2.4
jsonschema==4.20.0
jsonschema-specifications==2023.11.2
jupyter==1.0.0
jupyt

In [70]:
import arlpy.uwapm as pm
import arlpy.plot as plt
import numpy as np

env = pm.create_env2d()


# make the speed of sound profile for the ORCA site (ANTARES data used): https://www.sincem.unibo.it/images/tesi/tesi_Zanella.pdf
ssp = [
    [   0, 1516.91274542], 
    [   1, 1516.91274542], 
    [  40, 1513.24625848], 
    [  50, 1510.8198138], 
    [ 100, 1506.71243224], 
    [ 200, 1508.06772055], 
    [ 600, 1514.23874969], 
    [ 800, 1517.31426587], 
    [1000, 1520.32601248], 
    [2000, 1536.36730689], 
    [2475, 1544.35065635]]

# make the bathymetry for the environment: 1% slope Maarten described
bathy = [
    [0, 2475],       # 2475 m water depth at the transmitter
    [1000, 2465],    # 2465 m water depth 1000 m away
    [1100, 2465]]    # 2465 m water depth 1100 m away

# frequency min and max: https://www.sciencedirect.com/science/article/pii/S0168900212015239
frequency_min = 25000      # 25 kHz
frequency_max = 45000      # 45 kHz

In [75]:
# Create the 2D environment with both the source and the receiver placed 3 m above the sea floor
env_min = pm.create_env2d(frequency=frequency_min, min_angle=-89, max_angle=89, depth=bathy, tx_depth=2472, rx_depth=2462, rx_range=1000, soundspeed=ssp, nbeams=10000)
env_max = pm.create_env2d(frequency=frequency_max, min_angle=-89, max_angle=89, depth=bathy, tx_depth=2472, rx_depth=2462, rx_range=1000, soundspeed=ssp, nbeams=10000)

# Plot the sound speed profile
pm.plot_ssp(env_min, width=500)

In [3]:
pm.models()

['bellhop']

# Plotting an Environment

In [73]:
# Plotting an Environment using ARLPY
pm.plot_env(env_min, width=900) 

# Eigenrays

- **Eigenray** plots show just the rays that connect the source to a receiver.

In [76]:
# Eigenrays using ARLPY
rays_min = pm.compute_eigenrays(env_min)
rays_max = pm.compute_eigenrays(env_max)

with plt.figure(title='Acoustics KM3NeT ORCA', xlabel='Distance [m]', ylabel='Depth [m]', xlim=[-1, 1100], ylim=[-2500, -2000], width=900) as f:
    pm.plot_rays(rays_min, env=env_min, width=900)
    pm.plot_rays(rays_max, env=env_max, width=900)

In [29]:
# Rays using ARLPY
rays = pm.compute_rays(env)

In [32]:
with plt.figure(title='Acoustics KM3NeT ORCA', xlabel='Distance [m]', ylabel='Depth [m]', xlim=[-1, 1100], ylim=[-2500, 500], width=900) as f:
    pm.plot_rays(rays, env=env, width=900)

KeyboardInterrupt: 

In [78]:
arrivals_min = pm.compute_arrivals(env_min)
arrivals_max = pm.compute_arrivals(env_max)

arrivals_min[arrivals_min.arrival_number < 100][['time_of_arrival', 'angle_of_departure', 'angle_of_arrival', 'surface_bounces', 'bottom_bounces']]

,time_of_arrival,angle_of_departure,angle_of_arrival,surface_bounces,bottom_bounces
1,12.971779,-85.332832,88.773804,4,3
2,9.742387,-84.959000,-88.399506,3,3
3,9.738543,-84.941193,87.233696,3,2
4,6.513894,-83.588257,-85.882446,2,2
5,6.509998,-83.570458,84.716896,2,1
6,6.510001,-83.552658,84.701004,2,1
7,3.306285,-78.407944,-79.555367,1,1
8,3.302454,-78.390137,78.391602,1,0
9,0.647605,-0.261083,-0.884825,0,0
10,0.647609,-0.073328,-1.070036,0,1


In [79]:
arrivals_max[arrivals_max.arrival_number < 100][['time_of_arrival', 'angle_of_departure', 'angle_of_arrival', 'surface_bounces', 'bottom_bounces']]

,time_of_arrival,angle_of_departure,angle_of_arrival,surface_bounces,bottom_bounces
1,12.971779,-85.332832,88.773804,4,3
2,9.742387,-84.959000,-88.399506,3,3
3,9.738543,-84.941193,87.233696,3,2
4,6.513894,-83.588257,-85.882446,2,2
5,6.509998,-83.570458,84.716896,2,1
6,6.510001,-83.552658,84.701004,2,1
7,3.306285,-78.407944,-79.555367,1,1
8,3.302454,-78.390137,78.391602,1,0
9,0.647605,-0.261083,-0.884825,0,0
10,0.647609,-0.073328,-1.070036,0,1


# Better Angle Selection?

In [102]:
# Create the 2D environment with both the source and the receiver placed 3 m above the sea floor
env_angle = pm.create_env2d(frequency=frequency_min, min_angle=-0.75, max_angle=0.25, depth=bathy, tx_depth=2472, rx_depth=2462, rx_range=1000, soundspeed=ssp, nbeams=10000)

In [100]:
arrivals_angle = pm.compute_arrivals(env_angle)
arrivals_angle[arrivals_angle.arrival_number < 100][['time_of_arrival', 'angle_of_arrival', 'surface_bounces', 'bottom_bounces']]

,time_of_arrival,angle_of_arrival,surface_bounces,bottom_bounces
1,0.647587,-0.770231,0,0
2,0.647587,-0.846566,0,1


In [84]:
# Eigenrays using ARLPY
rays_angle = pm.compute_eigenrays(env_angle)

with plt.figure(title='Acoustics KM3NeT ORCA', xlabel='Distance [m]', ylabel='Depth [m]', xlim=[-1, 1100], ylim=[-2500, -2000], width=900) as f:
    pm.plot_rays(rays_angle, env=env, width=900)

# Greater Depth?

In [43]:
env_depth = pm.create_env2d()


# make the speed of sound profile for the ORCA site (ANTARES data used): https://www.sincem.unibo.it/images/tesi/tesi_Zanella.pdf
ssp_depth = [
    [   0, 1516.91274542], 
    [   1, 1516.91274542], 
    [  40, 1513.24625848], 
    [  50, 1510.8198138], 
    [ 100, 1506.71243224], 
    [ 200, 1508.06772055], 
    [ 600, 1514.23874969], 
    [ 800, 1517.31426587], 
    [1000, 1520.32601248], 
    [2000, 1536.36730689], 
    [2475, 1544.35065635],
    [3400, 1561]]

# make the bathymetry for the environment: 1% slope Maarten described
bathy_depth = [
    [0, 3400],       # 3400 m water depth at the transmitter
    [1000, 3390],    # 3390 m water depth 1000 m away
    [1100, 3390]]    # 3390 m water depth 1100 m away

In [80]:
# Create the 2D environment with both the source and the receiver placed 3 m above the sea floor
env_depth = pm.create_env2d(frequency=frequency_min, min_angle=-89, max_angle=89, depth=bathy_depth, tx_depth=3397, rx_depth=3387, rx_range=1000, soundspeed=ssp_depth, nbeams=10000)

# Plot the sound speed profile
pm.plot_ssp(env_depth, width=500)

In [45]:
# Plotting an Environment using ARLPY
pm.plot_env(env_depth, width=900) 

In [46]:
# Eigenrays using ARLPY
rays_depth = pm.compute_eigenrays(env_depth)

with plt.figure(title='Acoustics KM3NeT ORCA', xlabel='Distance [m]', ylabel='Depth [m]', xlim=[-1, 1100], ylim=[-3500, 500], width=900) as f:
    pm.plot_rays(rays_depth, env=env_depth, width=900)

In [81]:
arrivals_depth = pm.compute_arrivals(env_depth)
arrivals_depth[arrivals_depth.arrival_number < 100][['time_of_arrival', 'angle_of_arrival', 'surface_bounces', 'bottom_bounces']]

,time_of_arrival,angle_of_arrival,surface_bounces,bottom_bounces
1,17.736994,89.559319,4,3
2,13.311288,-89.429634,3,3
3,13.307550,88.269897,3,2
4,8.888267,-87.429466,2,2
5,8.884397,86.268173,2,1
6,4.480165,-82.617416,1,1
7,4.476321,81.453468,1,0
8,0.640703,-0.925795,0,0
9,0.640702,-0.804972,0,1
10,0.640706,-1.088526,0,1


# Different Distance and Bathy

In [141]:
env_try = pm.create_env2d()


# make the speed of sound profile for the ORCA site (ANTARES data used): https://www.sincem.unibo.it/images/tesi/tesi_Zanella.pdf
ssp_try = [
    [   0, 1516.91274542], 
    [   1, 1516.91274542], 
    [  40, 1513.24625848], 
    [  50, 1510.8198138], 
    [ 100, 1506.71243224], 
    [ 200, 1508.06772055], 
    [ 600, 1514.23874969], 
    [ 800, 1517.31426587], 
    [1000, 1520.32601248], 
    [2000, 1536.36730689], 
    [2475, 1544.35065635],
    [3400, 1561]]

# make the bathymetry for the environment: 2% slope found in Git release: 751cf5f
bathy_try = [
    [0, 3400],       # 3400 m water depth at the transmitter
    [1100, 3390],    # 3376 m water depth 1500 m away
    [1200, 3390]]    # 3376 m water depth 1600 m away

# frequency min and max: https://www.sciencedirect.com/science/article/pii/S0168900212015239
frequency_min = 25000      # 25 kHz
frequency_max = 45000      # 45 kHz

In [142]:
# Create the 2D environment with both the source and the receiver placed 3 m above the sea floor
env_try = pm.create_env2d(frequency=frequency_min, min_angle=-89, max_angle=89, depth=bathy_try, tx_depth=3398, rx_depth=3388, rx_range=1100, soundspeed=ssp_try, nbeams=10000)

In [143]:
arrivals_try = pm.compute_arrivals(env_try)
arrivals_try[arrivals_try.arrival_number < 100][['time_of_arrival', 'angle_of_arrival', 'surface_bounces', 'bottom_bounces']]

,time_of_arrival,angle_of_arrival,surface_bounces,bottom_bounces
1,17.738258,89.192100,4,3
2,13.314920,-88.940071,3,3
3,13.312401,87.882782,3,2
4,8.893369,-86.847611,2,2
5,8.890777,85.790085,2,1
6,4.490133,-81.676979,1,1
7,4.487567,80.616783,1,0
8,0.704758,-0.910196,0,0
9,0.704758,-0.987204,0,1
10,4.490134,80.609383,1,1
